In [1]:
!pip install geopandas

In [2]:
import calitp
from calitp.tables import tbl
from siuba import *

import pandas as pd
import numpy as np
import geopandas as gpd

initial thoughts:

* For long trips (LA->San Jose, etc) 1 bus in the fleet per daily trip could be a reasonable assumption
* Medium trips (Stockton --> Sac, LA-->SD); .5 buses in fleet/trip?
* Short trips (SF Transbay), .3 buses in fleet/trip?

One validator per bus seems adequate since these are single-door, over-the-road coaches.

### Find stops for routes of interest (this seems to work fine)

In [3]:
amtrak_routes = (tbl.gtfs_schedule.routes()
                >> filter(_.calitp_itp_id == 13)
                >> collect())

In [4]:
ca_corridors = ['Capitol Corridor', 'Pacific Surfliner', 'San Joaquins']

In [5]:
ca_rail_routes = amtrak_routes >> filter(_.route_long_name.isin(ca_corridors))

In [6]:
ca_rail_routes = ca_rail_routes >> select(_.route_id, _.route_long_name)

In [7]:
ca_rail_trips = (tbl.gtfs_schedule.trips()
                >> filter(_.calitp_itp_id == 13)
                >> collect()
                >> inner_join(_, ca_rail_routes, on='route_id')
                )

In [8]:
ca_corridor_stops = (tbl.gtfs_schedule.stop_times()
                >> filter(_.calitp_itp_id == 13)
                >> filter(_.trip_id.isin(ca_rail_trips.trip_id))
                >> collect()
                >> distinct(_.stop_id, _keep_all = True)
                )

In [9]:
ca_corridor_stops.head(3)

,calitp_itp_id,calitp_url_number,trip_id,stop_id,stop_sequence,arrival_time,departure_time,stop_headsign,pickup_type,drop_off_type,continuous_pickup,continuous_drop_off,shape_dist_traveled,timepoint,calitp_extracted_at
0,13,0,7012818749,BFD,1,9:12:00,9:12:00,None,0,0,None,None,None,None,2021-10-19
1,13,0,5212816183,SAC,1,7:10:00,7:10:00,None,0,0,None,None,None,None,2021-10-19
2,13,0,7202808666,OKJ,1,10:12:00,10:12:00,None,0,0,None,None,None,None,2021-10-19


In [10]:
ca_thruway_trips = (tbl.gtfs_schedule.routes()
                >> filter(_.calitp_itp_id == 13)
                >> filter(_.route_long_name == 'Amtrak Thruway Connecting Service')
                >> inner_join(_, tbl.gtfs_schedule.trips(), on='route_id')
                >> inner_join(_, tbl.gtfs_schedule.stop_times(), on='trip_id')
                >> filter(_.stop_id.isin(ca_corridor_stops.stop_id)) ## filter to trips connecting with CCJPA, LOSSAN and SJRRA
                >> collect()
                >> distinct(_.trip_id, _keep_all=True)
                # >> select(_.calitp_itp_id, _.trip_id, _.route_id, _.service_id , _.shape_id) ## no shape_id
                >> select(_.calitp_itp_id, _.trip_id, _.route_id, _.service_id ,))

In [11]:
ca_thruway_trips

,calitp_itp_id,trip_id,route_id,service_id
0,13,49842815638,37329,2815638
1,13,54162815345,37329,2815345
2,13,66472814258,37329,2814258
3,13,66472814256,37329,2814256
4,13,54122815340,37329,2815340
...,...,...,...,...
858,13,66772814212,37329,2814212
859,13,66772814211,37329,2814211
860,13,33292814231,37329,2814231
861,13,50112817980,37329,2817980


In [12]:
amtrak_thursday_service = (
    tbl.views.gtfs_schedule_fact_daily_service()
    >> filter(_.calitp_itp_id == 13)
    >> filter(_.service_date == '2021-10-28') ## a Thursday
    >> collect())

In [13]:
thursday_thruway_trips = ca_thruway_trips >> filter(_.service_id.isin(amtrak_thursday_service.service_id))

In [14]:
thursday_thruway_trips.shape

(400, 4)

In [15]:
thursday_thruway_trips.head(10)

,calitp_itp_id,trip_id,route_id,service_id
1,13,54162815345,37329,2815345
2,13,66472814258,37329,2814258
5,13,63112816851,37329,2816851
11,13,54102815349,37329,2815349
21,13,66112814305,37329,2814305
28,13,54122815339,37329,2815339
29,13,36242819614,42917,2819614
33,13,40152820008,42895,2820008
34,13,49672819218,37329,2819218
38,13,63132816849,37329,2816849


In [16]:
thursday_thruway_trips['route_id'].value_counts()

37329    380
11317      8
42896      6
42895      4
42917      2
Name: route_id, dtype: int64

#### 380 daily trips for route_id 37329 seems like an error. Also that ID seems to serve many routes?

In [45]:
thursday_thruway_stop_times = (tbl.gtfs_schedule.stop_times()
                               >> filter(_.calitp_itp_id == 13)
                               >> filter(_.trip_id.isin(thursday_thruway_trips.trip_id))
                               >> collect()
                               >> inner_join(_, thursday_thruway_trips, on='trip_id'))

## Seems to be way off (San Jose and Stockton are not served hundreds of times/day by Thruway buses...)

In [46]:
thursday_trips_by_stop = thursday_thruway_stop_times >> count(_.route_id, _.stop_id, sort=True)
thursday_trips_by_stop

,route_id,stop_id,n
0,37329,SJC,274
1,37329,SKN,199
2,37329,FRT,182
3,37329,TRA,182
4,37329,DBP,182
...,...,...,...
134,37329,SIM,1
135,37329,CPN,1
136,37329,CML,1
137,37329,SNC,1


In [47]:
thursday_trips_by_stop >> filter(_.stop_id.isin(['LAX', 'BFD', 'SLV', 'SJC']))

,route_id,stop_id,n
0,37329,SJC,274
12,37329,SLV,92
20,37329,BFD,32
22,37329,LAX,21
34,11317,SJC,8
74,42896,LAX,4
93,42895,SJC,2
102,42896,BFD,2
129,42895,LAX,2


In [20]:
thursday_trips_by_stop >> filter(_.stop_id == 'EMY')

,route_id,stop_id,n
7,37329,EMY,125


In [21]:
thursday_trips_by_stop >> filter(_.stop_id == 'PSP')

,route_id,stop_id,n
84,37329,PSP,4


In [22]:
amtrak_stops = tbl.gtfs_schedule.stops() >> filter(_.calitp_itp_id == 13) >> collect()
thursday_trips_by_stop = thursday_trips_by_stop >> inner_join(_, amtrak_stops, on='stop_id')
thursday_trips_by_stop = thursday_trips_by_stop >> filter(_.route_id == '37329')
thursday_trips_by_stop = gpd.GeoDataFrame(thursday_trips_by_stop, 
                            geometry=gpd.points_from_xy(thursday_trips_by_stop.stop_lon, thursday_trips_by_stop.stop_lat),
                            crs='EPSG:4326')
    

### Sandbox

In [23]:
!pip install ipyleaflet

E1022 21:53:31.282047100     262 backup_poller.cc:133]       Run client channel backup poller: {"created":"@1634939611.281913349","description":"pollset_work","file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":320,"referenced_errors":[{"created":"@1634939611.281909118","description":"Bad file descriptor","errno":9,"file":"src/core/lib/iomgr/ev_epollex_linux.cc","file_line":950,"os_error":"Bad file descriptor","syscall":"epoll_wait"}]}
E1022 21:53:31.327127701     268 ev_epollex_linux.cc:515]    Error shutting down fd 72. errno: 9


In [39]:
from ipyleaflet import Map, GeoJSON, projections, basemaps, GeoData, LayersControl, WidgetControl, GeoJSON
from ipywidgets import Text, HTML

def map_gdf(gdf, mouseover=None):
    
    if 'calitp_extracted_at' in gdf.columns:
        gdf = gdf.drop(columns='calitp_extracted_at')
    gdf = gdf.to_crs('EPSG:6414') ## https://epsg.io/6414 (meters)
    if gdf.geometry.iloc[0].geom_type == 'Point':
        gdf.geometry = gdf.geometry.buffer(4000)
    
    x = gdf.to_crs('EPSG:4326').geometry.iloc[0].centroid.x
    y = gdf.to_crs('EPSG:4326').geometry.iloc[0].centroid.y
    
    m = Map(basemap=basemaps.CartoDB.Positron, center=[y, x], zoom=11)

    if mouseover:
        html = HTML(f'hover to see {mouseover} Trips/day?')
        html.layout.margin = '0px 20px 20px 20px'
        control = WidgetControl(widget=html, position='topright')
        m.add_control(control)

        def update_html(feature,  **kwargs):
            html.value = '''
                <h3><b>{} Trips/day?</b></h3>
            '''.format(feature['properties'][mouseover])
    

    geo_data = GeoData(geo_dataframe = gdf.to_crs('EPSG:4326'),
                           style={'color': 'black', 'fillColor': '#253494',
                                        'opacity':0.4, 'weight':.5, 'dashArray':'2', 'fillOpacity':0.7},
                           hover_style={'fillColor': 'red' , 'fillOpacity': 0.9},
                           name = 'gdf')
    m.add_layer(geo_data)
    
    if mouseover:
        geo_data.on_hover(update_html)

    m.add_control(LayersControl())

    return m

In [42]:
tbl.gtfs_schedule.trips() >> filter(_.calitp_itp_id == 13)

,calitp_itp_id,calitp_url_number,route_id,service_id,trip_id,shape_id,trip_headsign,trip_short_name,direction_id,block_id,wheelchair_accessible,bikes_allowed,calitp_extracted_at
0,13,0,37329,2797402,39112797402,None,San Jose,3911,1,None,None,None,2021-10-19
1,13,0,37329,2797403,39112797403,None,San Jose,3911,1,None,None,None,2021-10-19
2,13,0,37329,2797416,47962797416,None,San Luis Obispo,4796,1,None,None,None,2021-10-19
3,13,0,37329,2797417,47962797417,None,San Luis Obispo,4796,1,None,None,None,2021-10-19
4,13,0,37329,2797479,39152797479,None,San Jose,3915,1,None,None,None,2021-10-19


In [41]:
tbl.gtfs_schedule.shapes() >> filter(_.calitp_itp_id == 13)

,calitp_itp_id,calitp_url_number,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,calitp_extracted_at
0,13,0,3,44.521053,-73.394035,315,None,2021-10-19
1,13,0,3,43.811103,-73.394451,827,None,2021-10-19
2,13,0,3,42.258361,-73.795262,1644,None,2021-10-19
3,13,0,4,42.787777,-73.939269,1244,None,2021-10-19
4,13,0,4,43.208691,-73.635209,1501,None,2021-10-19


In [40]:
map_gdf(thursday_trips_by_stop, 'n')

Map(center=[37.32993504325185, -121.90238799999997], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [26]:
(tbl.gtfs_schedule.trips()
                >> filter(_.calitp_itp_id == 13)
                >> count(_.route_id))

,route_id,n
0,37329,917
1,88,360
2,71,218
3,40751,199
4,84,169


In [27]:
route_37329 = (tbl.gtfs_schedule.routes()
                >> filter(_.calitp_itp_id == 13)
                >> filter(_.route_long_name == 'Amtrak Thruway Connecting Service')
                >> collect())

In [28]:
(tbl.gtfs_schedule.shapes()
                >> filter(_.calitp_itp_id == 13))

,calitp_itp_id,calitp_url_number,shape_id,shape_pt_lat,shape_pt_lon,shape_pt_sequence,shape_dist_traveled,calitp_extracted_at
0,13,0,3,44.521053,-73.394035,315,None,2021-10-19
1,13,0,3,43.811103,-73.394451,827,None,2021-10-19
2,13,0,3,42.258361,-73.795262,1644,None,2021-10-19
3,13,0,4,42.787777,-73.939269,1244,None,2021-10-19
4,13,0,4,43.208691,-73.635209,1501,None,2021-10-19


In [29]:
ca_thruway_trips >> filter(_.service_id.isin(['2814305']))

,calitp_itp_id,trip_id,route_id,service_id
21,13,66112814305,37329,2814305


In [30]:
df = (tbl.views.gtfs_schedule_fact_daily_service()
    >> filter(_.calitp_itp_id == 13)
    >> filter(_.service_date == '2021-10-28') ## a Thursday
    >> inner_join(_, tbl.gtfs_schedule.trips() >> filter(_.calitp_itp_id == 13), on = 'service_id')
    >> filter(_.route_id == '37329') ## this route_id seems to encompass many thruway bus trips in CA
    >> collect())
df

,feed_key,calitp_itp_id_x,calitp_url_number_x,service_date,service_id,ttl_service_hours,n_trips,n_routes,first_departure_ts,last_arrival_ts,...,wheelchair_accessible,calitp_url_number_y,block_id,bikes_allowed,route_id,trip_headsign,direction_id,calitp_extracted_at,trip_short_name,calitp_itp_id_y
0,-1246851374109274803,13,0,2021-10-28,2813905,2.000000,1,1,54900,62100,...,None,0,None,None,37329,Bellingham Amtrak Station,0,2021-10-19,5622,13
1,-1246851374109274803,13,0,2021-10-28,2814189,2.000000,1,1,43500,50700,...,None,0,None,None,37329,Stockton,1,2021-10-19,3812,13
2,-1246851374109274803,13,0,2021-10-28,2819224,2.000000,1,1,58200,65400,...,None,0,None,None,37329,Stockton,1,2021-10-19,3816,13
3,-1246851374109274803,13,0,2021-10-28,2814174,2.250000,1,1,33000,41100,...,None,0,None,None,37329,Stockton,0,2021-10-19,3910,13
4,-1246851374109274803,13,0,2021-10-28,2818589,2.250000,1,1,30000,38100,...,None,0,None,None,37329,Martinez,1,2021-10-19,6510,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
412,-1246851374109274803,13,0,2021-10-28,2817681,1.416667,1,1,74700,79800,...,None,0,None,None,37329,Santa Rosa Amtrak Bus Stop,0,2021-10-19,6515,13
413,-1246851374109274803,13,0,2021-10-28,2819231,1.666667,1,1,67800,73800,...,None,0,None,None,37329,Martinez,1,2021-10-19,6518,13
414,-1246851374109274803,13,0,2021-10-28,2819214,1.916667,1,1,85500,92400,...,None,0,None,None,37329,Davis,0,2021-10-19,3819,13
415,-1246851374109274803,13,0,2021-10-28,2813788,3.716667,1,1,20400,33780,...,None,0,None,None,37329,Milwaukee Airport Amtrak Bus Stop,1,2021-10-19,6332,13


In [31]:
amtrak_thursday_service

,feed_key,calitp_itp_id,calitp_url_number,service_date,service_id,ttl_service_hours,n_trips,n_routes,first_departure_ts,last_arrival_ts,service_window
0,-1246851374109274803,13,0,2021-10-28,2813905,2.000000,1,1,54900,62100,2.000000
1,-1246851374109274803,13,0,2021-10-28,2814189,2.000000,1,1,43500,50700,2.000000
2,-1246851374109274803,13,0,2021-10-28,2815144,2.000000,1,1,48000,55200,2.000000
3,-1246851374109274803,13,0,2021-10-28,2819224,2.000000,1,1,58200,65400,2.000000
4,-1246851374109274803,13,0,2021-10-28,2819454,2.000000,1,1,86400,93600,2.000000
...,...,...,...,...,...,...,...,...,...,...,...
1012,-1246851374109274803,13,0,2021-10-28,2809946,2.866667,1,1,64680,75000,2.866667
1013,-1246851374109274803,13,0,2021-10-28,2808673,3.116667,1,1,72300,83520,3.116667
1014,-1246851374109274803,13,0,2021-10-28,2820157,5.983333,1,1,32100,53640,5.983333
1015,-1246851374109274803,13,0,2021-10-28,2809400,7.983333,1,1,55200,83940,7.983333


In [32]:
tbl.views.gtfs_schedule_fact_daily_trips() >> filter(_.calitp_itp_id == 13)

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 500 Query exceeded limit for bytes billed: 5000000000. 10208935936 or higher required.

(job ID: 701bedd9-efd5-4cb1-b82f-0ae586c9bd3f)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          

DatabaseError: (google.cloud.bigquery.dbapi.exceptions.DatabaseError) 500 Query exceeded limit for bytes billed: 5000000000. 10208935936 or higher required.

(job ID: c37e8a2f-e4f5-4e25-8b2f-70bfcf6a2644)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          